In [ ]:

%run includes/1-res-heirarchy-setup.ipynb

# GCP Resources
Summary of the GCP Resource Hierarchy including folders and projects.

In [ ]:

# discover folders and projects
def get_projects_query(entity_id):
    return f"""
    SELECT displayName, name, parent, projectId 
    FROM google.cloudresourcemanager.projects 
    WHERE parent = '{entity_id}';
    """

def get_folders_query(entity_id):
    return f"""
    SELECT displayName, name, parent 
    FROM google.cloudresourcemanager.folders 
    WHERE parent = '{entity_id}';
    """

tree, resources_df, all_projects, all_project_numbers, cards_data = get_all_resources(get_projects_query, get_folders_query, org_id.value)

## Folder and Project Summary

In [ ]:

# summary cards
display_cards(cards_data)

## Resource Hierarchy

In [ ]:

# show res heirarchy
tree

## Enabled Services by Project

In [ ]:
# get service usage across all projects
serviceusage_queries = [
    f"""
    select 
    '{project}' as project,
    split_part(split_part(name, '/', -1), '.', 1) as service
    from google.serviceusage.services
    where parent = '{project}' and parentType = 'projects' and filter = 'state:ENABLED'
    """
    for project in all_projects
]
serviceusage_df = run_stackql_queries(serviceusage_queries)

In [ ]:

# service usage summary
plot_serviceusage(serviceusage_df)

In [ ]:

# save data
save_dataframe(resources_df, 'resources_df')
save_object(all_projects, 'all_projects')
save_object(all_project_numbers, 'all_project_numbers')